In [10]:
import math
import random

import networkit as nk
import torch.distributions
from networkit import Graph
from torch_cluster import random_walk
from torch_geometric import EdgeIndex
from torch_geometric.utils import from_networkit


class PPR:
    def __init__(self, G: Graph, alpha, delta):
        n = G.numberOfNodes()
        is_undir = not G.isDirected()
        G_pyg = from_networkit(G)[0]
        G_pyg = EdgeIndex(G_pyg, sparse_size=(n, n), is_undirected=is_undir)
        self._G = G_pyg
        self._G_nk = G_nk
        self._alpha = alpha
        self._delta = delta
        self.eps = math.sqrt(delta)
        self.c = 350
        self.beta = 1 / 6

    def fast_ppr(self, s, t):
        t_set, f_set, pi_inv = self._frontier(t)
        if s in t_set:
            return pi_inv[s]
        else:
            k = round(self.c * self.eps / self._delta)
            H = [None] * k
            for i in range(k):
                L = int(torch.distributions.Geometric(torch.tensor([self._alpha])).sample().int()) + 1
                RW = random_walk(self._G[0], self._G[1], torch.tensor([s]), L)[0]
                for node in RW:
                    node = int(node)
                    if node in f_set:
                        H[i] = node
            return (1 / k) * sum([pi_inv[H[i]] if H[i] is not None else 0 for i in range(k)]), t_set, f_set

    def _frontier(self, t):
        eps_inverse = self.beta * self.eps
        pi = [0 if u != t else self._alpha for u in self._G_nk.iterNodes()]
        r = [0 if u != t else self._alpha for u in self._G_nk.iterNodes()]
        target_set = {t}
        frontier_set = set()
        while any(r[w] > self._alpha * eps_inverse for w in self._G_nk.iterNodes()):
            for w in self._G_nk.iterNodes():
                if r[w] > self._alpha * eps_inverse:
                    for u in self._G_nk.iterInNeighbors(w):
                        Delta = (1 - self._alpha) * (r[w] / self._G_nk.degreeOut(u))
                        pi[u] = pi[u] + Delta
                        r[u] = r[u] + Delta
                        if pi[u] > self.eps:
                            target_set.add(u)
                            for n in self._G_nk.iterInNeighbors(u):
                                frontier_set.add(n)
                r[w] = 0
        frontier_set = frontier_set.difference(target_set)
        return target_set, frontier_set, pi

In [12]:
G_nk = nk.readGraph("./foodweb-baydry.konect")
delta = 1 / G_nk.numberOfNodes()
my_ppr = PPR(G_nk, alpha=0.3, delta=delta)
t_sizes = []
f_sizes = []
below_delta = 0
for i in range(1, 20):
    start = random.randint(0, G_nk.numberOfNodes() - 1)
    target = random.randint(0, G_nk.numberOfNodes() - 1)
    result, target_set, frontier_set = my_ppr.fast_ppr(start, target)
    t_sizes.append(len(target_set))
    f_sizes.append(len(frontier_set))
    if result < delta:
        below_delta += 1
print("Average size of target set: ", sum(t_sizes) / len(t_sizes))
print("Average size of frontier set: ", sum(f_sizes) / len(f_sizes))
print("Percentage of values below delta: ", below_delta / 20)


Average size of target set:  2.1578947368421053
Average size of frontier set:  7.7368421052631575
Percentage of values below delta:  0.9


The parameter alpha sets the probability of restarting the random walk at the origin node. A smaller value for alpha therefore leads to longer random walks (on average). 